## Shenick LOG分析脚本
---
**环境**：
- Python 3.5+
- pandas 0.18.1
- matlibplot 2.0.0
- numpy 1.11.3+mkl

**功能**
- 统计VoLTE/FTP/UDP信息

**限制**
- TODO

**历史**
- 0.01: 2017/07/27
- 0.02: 增加对VoLTE语音速率的统计

**TO DO**
- 增加对多DRB应用的支持
- 增加对ping应用的处理
- 更多验证

### 导入相关模块，定义Log文件路径和公共函数

In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

# Shenick Log所在路径
log_path = r"D:\userdata\anliu\Desktop\log\CP5\Shenick" 

file_path = os.path.join(log_path, "Application")  
try: 
    os.chdir(file_path)
except:
    print("The folder does not exist!")

### VoLTE统计信息

In [ ]:
def volte_process():
    # 读取Log文件，如果文件不存在返回打印信息
    volte_logs = glob.glob("./cvoip*.csv")
    if volte_logs:
        print("Start to analyze VoLTE log...")
    else:
        print("no VoLTE log!")
        return  
    
    # 统计VoLTE UE ID，MOS值，Jitter值，Packet Loss Rate（PLR）和Volte失败的UE数，以及主被叫收发包速率即AMR-WB/AMR-NB的速率
    volte_total_num = 0
    # 为了可视化添加ue_id列表
    volte_ue_id = [] 
    volte_mos = []
    volte_jitter = []
    volte_plr = []
    volte_delay = []
    volte_fail_ue_id = []
    volte_caller_out_rate = []
    volte_caller_in_rate = []
    volte_callee_out_rate = []
    volte_callee_in_rate = []
    
    for log in volte_logs:
        p = re.search(r'_(\d+)_', log)
        ue_id = p.group(1)
        
        df_log = pd.read_csv(log, low_memory=False, skip_blank_lines=True, na_values=['nan']) 
        # 如果In Service值的和等于0，说明该VoLTE应用没有被调用，不统计
        if (df_log['In Service'].sum() == 0):
            continue
        volte_total_num += 1
        volte_ue_id.append(ue_id)
        # 尽量避免使用dataframe的hard copy，这是一件非常占用资源和时间的做法，可以对比下面code和上面的code，实现类似的功能，下面的做法需要花费更多的时间
#         df_log = df_log[df_log['In Service']==1]
#         if df_log.empty:
#             continue
#         volte_total_num += 1

        # 'RTP Mean Latency ms'统计开关不是默认开启。如果开关没有开启，增加'RTP Mean Latency ms'列并取值为0
        if ('RTP Mean Latency ms') not in df_log:
            df_log['RTP Mean Latency ms'] = 0        
        df_log.dropna(axis=0, subset=['QmVoice MOS','QmVoice Discarded Packets', 'QmVoice In Packets','RTP Jitter (RFC3550) ms'], inplace=True)
        
        # 'RTP Mean Latency ms'统计只对主叫有效
        if (int(ue_id)%2) == 0:
            delay = df_log['RTP Mean Latency ms'].mean()
            if np.isnan(delay):
                delay = 0
            volte_delay.append(delay)

            # 主叫语音速率               
            caller_out_rate = df_log['Out RTP Bits/s'].mean()
            caller_in_rate = df_log['In RTP Bits/s'].mean()            
            volte_caller_out_rate.append(caller_out_rate)
            volte_caller_in_rate.append(caller_in_rate)
        else:
            callee_out_rate = df_log['Out RTP Bits/s'].mean()
            callee_in_rate = df_log['In RTP Bits/s'].mean()
            # 被叫语音速率
            volte_callee_out_rate.append(callee_out_rate)
            volte_callee_in_rate.append(callee_in_rate)
        
        # MOS/Jitter/PLR
        mos = df_log['QmVoice MOS'].mean()
        jitter = df_log["RTP Jitter (RFC3550) ms"].mean()
        plr = df_log['QmVoice Discarded Packets'].mean()/df_log['QmVoice In Packets'].mean()*100
        
        # 如果MOS值不存在，说明VoLTE call失败
        if np.isnan(mos): 
            volte_fail_ue_id.append(ue_id)
        
        volte_mos.append(mos)
        volte_jitter.append(jitter)
        volte_plr.append(plr)

    
    # 统计平均值的时候去掉VoLTE失败的UE
    mos_avg = np.nanmean(volte_mos)
    jitter_avg = np.nanmean(volte_jitter)
    plr_avg = np.nanmean(volte_plr)
    delay_avg = np.nanmean(volte_delay)
    
    caller_out_rate_avg = np.nanmean(volte_caller_out_rate)
    caller_in_rate_avg = np.nanmean(volte_caller_in_rate)
    callee_out_rate_avg = np.nanmean(volte_callee_out_rate)
    callee_in_rate_avg = np.nanmean(volte_callee_in_rate)

    volte_fail_num = len(volte_fail_ue_id)
    
    print("VoLTE total number:", volte_total_num)
    print("VoLTE fail number:", volte_fail_num)
    print("VoLTE failed ue id:", volte_fail_ue_id)
    
    print("VoLTE caller out rate:{:.1f}bps".format(caller_out_rate_avg))
    print("VoLTE caller in rate:{:.1f}bps".format(caller_in_rate_avg))    
    print("VoLTE callee out rate:{:.1f}bps".format(callee_out_rate_avg))  
    print("VoLTE callee in rate:{:.1f}bps".format(callee_in_rate_avg))     
    
    print("MOS: {:.2f}".format(mos_avg))
    print("Jitter: {:.2f}ms".format(jitter_avg))
    print("PLR: {:.2f}%".format(plr_avg))
    print("Delay: {:.2f}ms".format(delay_avg))
    
    # 可视化
    fig = plt.figure(1,(18,6))
    ax1 = fig.add_subplot(121)
    ax1.plot(volte_mos, 'g-',  label='MOS')
    ax1.set_title('MOS and PLR')
    ax1.set_ylabel('MOS')
    ax1.legend(loc=2)
    ax2 = ax1.twinx()
    ax2.plot(volte_plr, 'r--', label='PLR(%)')
    ax2.set_ylabel('PLR(%)')
    ax2.legend(loc=1)
    
    ax3 = fig.add_subplot(122)
    ax3.plot(volte_plr, volte_mos, 'c+')
    ax3.set_title('MOS/PLR')    
    ax3.set_xlabel('PLR(%)')
    ax3.set_ylabel('MOS')
    plt.show()

In [ ]:
volte_process()

### FTP统计信息

In [ ]:
# 获取取Log文件列表
def get_ftp_logs():
    ftp_get_logs = glob.glob("./cftp_get_[0-9]*.csv")
    ftp_put_logs = glob.glob("./cftp_put_[0-9]*.csv")
    if not ftp_get_logs:
        ftp_get_logs = []        
    if not ftp_put_logs:
        ftp_put_logs = []        
        
    return ftp_get_logs, ftp_put_logs

# FTP LOG文件处理公共函数
def ftp_log_process(ftp_logs):
    ftp_total_num = 0  
    # 为了可视化添加ue_id列表
    ftp_ue_id = []     
    ftp_fail_ue_id = []
    ftp_out_rate = []
    ftp_in_rate = []
    
    for ftp_log in ftp_logs:
        p = re.search(r'_(\d+)_', ftp_log)
        ue_id = p.group(1) 

        df_log = pd.read_csv(ftp_log, low_memory=False, skip_blank_lines=True, na_values=['nan'])     
        if (df_log['In Service'].sum() == 0):
            continue
        ftp_total_num += 1  
        ftp_ue_id.append(ue_id)        
               
        df_log.dropna(axis=0, subset=['Out Bits/s', 'In Bits/s'], inplace=True)        

        out_rate = df_log['Out Bits/s'].mean()
        in_rate = df_log['In Bits/s'].mean()
        
        # 如果FTP OUT/IN值不存在，说明UDP失败
        if np.isnan(out_rate) and np.isnan(in_rate): 
            ftp_fail_ue_id.append(ue_id)
            
        ftp_out_rate.append(out_rate)
        ftp_in_rate.append(in_rate)
     
    ftp_fail_num = len(ftp_fail_ue_id)
    ftp_out_rate_avg = np.nanmean(ftp_out_rate)
    ftp_in_rate_avg = np.nanmean(ftp_in_rate)
    
    return [ftp_total_num, ftp_fail_num, ftp_ue_id, ftp_fail_ue_id, ftp_out_rate_avg, ftp_in_rate_avg]

# 处理FTP GET/PUT文件    
def ftp_process():
    ftp_get_logs, ftp_put_logs = get_ftp_logs()
    
    if ftp_get_logs == []:
        print("No ftp get logs")
    else:
        ftp_get_process(ftp_get_logs)
        
    if ftp_put_logs == []:
        print("No ftp put logs")
    else:
        ftp_put_process(ftp_put_logs)

# 处理FTP GET文件
def ftp_get_process(ftp_get_logs):
    print("Start to analyze ftp get logs...")

    ftp_get_result_list = ftp_log_process(ftp_get_logs)
    
    print("FTP GET total number:", ftp_get_result_list[0])
    print("FTP GET fail number:", ftp_get_result_list[1])
    print("FTP GET failed ue id:", ftp_get_result_list[3])

    print("FTP GET out rate: {:.2f}bps".format(ftp_get_result_list[4]))
    print("FTP GET in rate: {:.2f}bps".format(ftp_get_result_list[5]))    
    
# 处理FTP PUT文件
def ftp_put_process(ftp_put_logs):
    print("Start to analyze ftp put logs...")

    ftp_put_result_list = ftp_log_process(ftp_get_logs)
    
    print("FTP PUT total number:", ftp_put_result_list[0])
    print("FTP PUT fail number:", ftp_put_result_list[1])
    print("FTP PUT failed ue id:", ftp_put_result_list[3])

    print("FTP PUT out rate: {:.2f}bps".format(ftp_put_result_list[4]))
    print("FTP PUT in rate: {:.2f}bps".format(ftp_put_result_list[5]))   

In [ ]:
ftp_process()

### UDP统计信息

In [ ]:
# 获取取Log文件列表
def get_udp_logs():
    dl_udp_logs = glob.glob("./DLUDP_[0-9]*.csv")
    ul_udp_logs = glob.glob("./ULUDP_[0-9]*.csv")
    if not dl_udp_logs:
        dl_udp_logs = []
    if not ul_udp_logs:
        ul_udp_logs = []

    return dl_udp_logs, ul_udp_logs
    
# 分别处理DL/UL UDP LOG文件    
def udp_process():
    dl_udp_logs, ul_udp_logs = get_udp_logs()
    
    if dl_udp_logs == []:
        print("No dl udp logs")
    else:
        dl_udp_process(dl_udp_logs)
        
    if ul_udp_logs == []:
        print("No ul udp logs")
    else:
        ul_udp_process(ul_udp_logs)
    
# UDP LOG文件处理公共函数
def udp_log_process(udp_logs):
    # udp发起数
    udp_total_num = 0  
    # 为了可视化添加ue_id列表
    udp_ue_id = []    
    udp_fail_ue_id = []
    udp_rate = []       
    
    for udp_log in udp_logs:
        p = re.search(r'_(\d+)_', udp_log)
        ue_id = p.group(1)       

        df_log = pd.read_csv(udp_log, low_memory=False, skip_blank_lines=True, na_values=['nan'])     
        if (df_log['In Service'].sum() == 0):
            continue
        udp_total_num += 1 
        udp_ue_id.append(ue_id)
               
        df_log.dropna(axis=0, subset=['Out Bits/s'], inplace=True)        

        out_rate = df_log['Out Bits/s'].mean()
        
        # 如果UDP OUT值不存在，说明UDP失败
        if np.isnan(out_rate): 
            udp_fail_ue_id.append(ue_id)
            
        udp_rate.append(out_rate)
     
    udp_fail_num = len(udp_fail_ue_id)
    udp_rate_avg = np.nanmean(udp_rate)
    
    return [udp_total_num, udp_fail_num, udp_ue_id, udp_fail_ue_id, udp_rate_avg]
    
# 处理DL UDP文件
def dl_udp_process(dl_udp_logs):
    print("Start to analyze dl udp logs...")
    
    dl_udp_result_list = udp_log_process(dl_udp_logs)
    
    print("DL UDP total number:", dl_udp_result_list[0])
    print("DL UDP fail number:", dl_udp_result_list[1])
    print("DL UDP failed ue id:", dl_udp_result_list[3])
    print("DL UDP Rate: {:.2f}bps".format(dl_udp_result_list[4]))
    
# 处理UL UDP文件
def ul_udp_process(ul_udp_logs):
    print("Start to analyze ul udp logs...")
    
    ul_udp_result_list = udp_log_process(ul_udp_logs)
    
    print("UL UDP total number:", ul_udp_result_list[0])
    print("UL UDP fail number:", ul_udp_result_list[1])
    print("UL UDP failed ue id:", ul_udp_result_list[3])
    print("UL UDP Rate: {:.2f}bps".format(ul_udp_result_list[4]))

In [ ]:
udp_process()